# Complex

In [1]:
import nest_asyncio
nest_asyncio.apply()

from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet

import time
import xrpl

from xrpl.models.transactions import Payment, Memo
from xrpl.transaction import safe_sign_and_autofill_transaction
import binascii
from xrpl.transaction import send_reliable_submission

import PySimpleGUI as sg

In [2]:
# create a network client
client = JsonRpcClient("https://s.altnet.rippletest.net:51234")
# Set up middle organization's XRPL account
org_account = generate_faucet_wallet(client)
# Set up voter database (can be replaced with a more secure storage solution)
voter_db = {
    'john': {
        'voted': False,
        'otp': '1234',
        'last_vote_time': 0
    },
    'jane': {
        'voted': False,
        'otp': '5678',
        'last_vote_time': 0
    }
}
for i in voter_db:
    user_wallet = generate_faucet_wallet(client)
    voter_db[i]["wallet"] = user_wallet

#create candidates
candidates_votes ={'ALICE': 0, 'BOB': 0, 'CHARLIE': 0, 'NONE': 0}

In [3]:
# Set up time window for voting
#vote_start_time = 1679241600 # March 18th, 2022 00:00:00 UTC
#vote_end_time = 1679328000 # March 19th, 2022 00:00:00 UTC

vote_start_time = int(time.time()) # Current time in seconds since the epoch
vote_end_time = vote_start_time + 86400 

In [4]:
# Define function to check if the user is authenticated
def authenticate_user(username, otp):
    if username in voter_db and voter_db[username]['otp'] == otp:
        return True
    else:
        return False

In [5]:
# Define function to check if the user can vote
def can_vote(username):
    if time.time() >= vote_start_time and time.time() <= vote_end_time:
        if not voter_db[username]['voted']:
            if time.time() - voter_db[username]['last_vote_time'] >= 86400: # One day in seconds
                return True
            else:
                print('\nYou have already voted today. Please try again tomorrow.')
                return False
        else:
            print('\nYou have already voted. Thank you!')
            return False
    else:
        print('\nVoting is closed. Please try again next time.')
        return False


In [6]:
def decoding_string(string):
    vote = string.result["Memos"][0]["Memo"]["MemoData"]
    vote = binascii.unhexlify(vote.encode()).decode('utf8')
    vote = vote.upper()
    candidates_votes[vote] = candidates_votes[vote] + 1


In [7]:
def correct_vote (vote):
    if vote.upper() in candidates_votes.keys():
        return True 

In [8]:

def cast_vote(username, candidate):
    # Create a Payment transaction object with a memo
    candidate_encode = binascii.hexlify(candidate.encode()).decode()
    payment_tx = Payment(
        account=voter_db[username]["wallet"].classic_address,
        amount= "1",  # Amount to send (in drops)
        destination=org_account.classic_address,
        memos= [Memo(memo_data=candidate_encode)]

    )

    # Sign the transaction with the source account's secret
    signed_tx = safe_sign_and_autofill_transaction(payment_tx, voter_db[username]["wallet"], client)
    tx_response = send_reliable_submission(signed_tx, client)
    tx_response
    decoding_string(tx_response)
    
    #Stated the usrr that voted
    voter_db[username]["voted"]=True


In [16]:
def generate_vote():
    sg.theme('Default')   # Add a touch of color
    # All the stuff inside your window.
    layout = [  [sg.Text('Name'), sg.InputText()],
                [sg.Text('Password (OTP)'), sg.InputText()],
                [sg.Button('Ok'), sg.Button('Cancel')] ]

    # Create the Window
    window = sg.Window('Window Title', layout)
    # Event Loop to process "events" and get the "values" of the inputs
    correct_user = False
    while True:
        event, values = window.read()
        if event == "Ok":
            if authenticate_user(values[0], values[1]):
                if can_vote(values[0]):
                    username = values[0]
                    correct_user = True
                    break
                else:
                    layout =  [ [sg.Text('You have already voted, thank you.')],
                                [sg.Button('Close')]
                                ]
                    window.close()
                    window = sg.Window('Window Title', layout)
            else:
                layout = [  [sg.Text('Name'), sg.InputText()],
                [sg.Text('Password (OTP)'), sg.InputText()],
                [sg.Text('Invalid username or OTP!!. Please try again.')],
                [sg.Button('Ok'), sg.Button('Cancel')] ]
                window.close()
                window = sg.Window('User information', layout)
        if event == sg.WIN_CLOSED or event == 'Cancel' or event == "Close": # if user closes window or clicks cancel
            break

    window.close()

    voted_successful = False
    if correct_user == True:
        layout = [[sg.Text('Please enter the candidate you are voting for: ')],
                [sg.Text('Options: Alice, Charlie, Bob, Blank')],
                [sg.Text('candidate:'), sg.InputText()],
                [sg.Button('Ok'), sg.Button('Cancel')]]
        window = sg.Window('Candidate voting', layout)
        while True:
            event, values = window.read()
            if correct_vote(values[0]):
                window.close()
                cast_vote(username, values[0])
                voted_successful = True
                break

            else:
                layout = [[sg.Text('Please enter the candidate you are voting for: ')],
                [sg.Text('Options: Alice, Charlie, Bob, Blank')],
                [sg.Text('candidate:'), sg.InputText()],
                [sg.Text('Incorrect imput, please try again.')],
                [sg.Button('Ok'), sg.Button('Cancel')]]
                window.close()
                window = sg.Window('Candidate voting', layout)
                
            if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
                break
            
        
        window.close()

    if voted_successful == True:
        layout = [[sg.Text('Your vote has been cast. Thank you!')],
                [sg.Button('Close')]]
        window = sg.Window('Done', layout)
        while True:
            event, values = window.read()
            if event == sg.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
                break
            

    window.close()


In [17]:
def vote_count():
    sg.theme('Default')   # Add a touch of color
    alice = 'Alice:'+" "+ str(candidates_votes["ALICE"])
    charlie = 'Charlie:'+" "+ str(candidates_votes["CHARLIE"])
    bob = 'Bob:'+" "+ str(candidates_votes["BOB"])
    none = 'None:'+" "+ str(candidates_votes["NONE"])
    total = 'Total:'+" "+ str(candidates_votes["NONE"] + candidates_votes["BOB"] + candidates_votes["CHARLIE"] + candidates_votes["ALICE"])
    layout = [[sg.Text(alice)],
            [sg.Text(charlie)],
            [sg.Text(bob)],
            [sg.Text(none)],
            [sg.Text(total)],
            [sg.Button('Close')]]
    window = sg.Window('Votes count', layout,)
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
            break

    window.close()

In [18]:
org_account.classic_address

'rNR9VxuY5EATu9oRanqTiFxV1PCVpqoBCB'

In [19]:
generate_vote()

Window will be a boring gray. Try removing the theme call entirely
 You will get the default theme or the one set in global settings
If you seriously want this gray window and no more nagging, add  theme('DefaultNoMoreNagging')  or theme('Gray Gray Gray') for completely gray/System Defaults

You have already voted. Thank you!
Window will be a boring gray. Try removing the theme call entirely
 You will get the default theme or the one set in global settings
If you seriously want this gray window and no more nagging, add  theme('DefaultNoMoreNagging')  or theme('Gray Gray Gray') for completely gray/System Defaults


In [15]:
vote_count()

Window will be a boring gray. Try removing the theme call entirely
 You will get the default theme or the one set in global settings
If you seriously want this gray window and no more nagging, add  theme('DefaultNoMoreNagging')  or theme('Gray Gray Gray') for completely gray/System Defaults
